# Лабораторная работа №3.
## Реализация сверточной нейронной сети


In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf


In [2]:
#1. Реализуйте нейронную сеть с двумя сверточными слоями, и одним полносвязным с нейронами с кусочно-линейной функцией активации. Какова точность построенное модели?


required_train_size=200000
required_test_size=10000
labels={}

def loadImages(path):
    data = {}
    index = 0
    label_dirs = os.listdir(f"{path}/")
    for label in label_dirs:
        zeros = np.zeros((len(label_dirs)), dtype=np.byte)
        zeros[index] = 1
        labels[label] = zeros
        index += 1
        for image in os.listdir(f"{path}/{label}/"):
            try:
                if label not in data:
                    data[label] = []
                img = mpimg.imread(f"{path}/{label}/{image}")
                data[label].append(img.reshape(img.shape[0], img.shape[1], 1))
            except:
                pass
    return data

data_large = loadImages("large")
data_small = loadImages("small")

def prepareDataset(data):
    X = []
    y = []
    for k, v in data.items():
        X = X + v
        y = y + [labels[k]]*len(v)
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int32)

X, y = prepareDataset(data_large)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=required_test_size, train_size=required_train_size)

del X
del y
del data_large

X_test, y_test = prepareDataset(data_small)
del data_small


In [3]:
start_learning_rate=0.05
shape = X_train.shape[1]
batch_size = 16
num_labels = len(labels.items())
optimizer = tf.optimizers.SGD(start_learning_rate)

#Параметры сверточного слоя
padding = 1
filter = 5
stride = 2
deph = 16
neurons = 64
layers = [tf.nn.relu, tf.nn.relu, tf.nn.relu]

def get_final_size(input_size, padding, stride, filters):
    output = input_size
    for filter in filters:
        output = (output + 2 * padding - filter) / stride + 1
    return int(np.ceil(output))

# Подготовка dataset для обучения
tf_valid_dataset = tf.constant(X_valid)
tf_test_dataset = tf.constant(X_test)

dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.repeat().shuffle(X_train.shape[0]).batch(batch_size)

#Определяет веса и смещения для слоев
def prepare_weights_biases(input_number, output_number, neuron_number, padding, stride, *args):
    weights = []
    biases = []
    previous_deph = 1
    stddev=0.1

    for deph, filter in args:
        weights.append(tf.Variable(tf.random.truncated_normal([filter, filter, previous_deph, deph], stddev=stddev)))
        biases.append(tf.Variable(tf.zeros([deph])))
        previous_deph = deph

    final_image_size = get_final_size(input_number, padding, stride, map(lambda arg: arg[1], args))
    weights.append(tf.Variable(tf.random.truncated_normal([final_image_size * final_image_size * previous_deph, neuron_number], stddev=stddev)))
    biases.append(tf.Variable(tf.zeros([neuron_number])))

    weights.append(tf.Variable(tf.random.truncated_normal([neuron_number, output_number], stddev=stddev)))
    biases.append(tf.Variable(tf.zeros([output_number])))

    return weights, biases

weights, biases = prepare_weights_biases(shape, num_labels, neurons, padding, stride, [deph, filter], [deph, filter])


def calculate_prediction(tf_dataset, weigths, biases, layers):
    prev_layer = tf_dataset
    for index, layer in enumerate(layers[:-1]):
        prev_layer = layer(tf.nn.conv2d(prev_layer, weigths[index], strides=[1, 2, 2, 1], padding='SAME') + biases[index])

    shape = prev_layer.get_shape().as_list()
    reshape = tf.reshape(prev_layer, [shape[0], shape[1] * shape[2] * shape[3]])
    prev_layer = layers[-1](tf.matmul(reshape,  weigths[-2]) + biases[-2])

    return tf.matmul(prev_layer, weigths[-1]) + biases[-1]

def run_optimization(tf_train_dataset, tf_train_labels, optimizer, weights, biases, layers):
    def cost():
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=calculate_prediction(tf_train_dataset, weights, biases, layers), labels=tf_train_labels))
    optimizer.minimize(cost, weights + biases, tape=tf.GradientTape())

def train_model(num_steps, weights, biases, optimizer, layers, optimization_function, calculate_prediction_function, tf_last_activation_function=tf.nn.softmax):
    for step, (batch_x, batch_y) in enumerate(dataset.take(num_steps), 1):
        optimization_function(batch_x, batch_y, optimizer, weights, biases, layers)
        if step % 2000 == 0 or step < 10:
            prediction = tf_last_activation_function(calculate_prediction_function(tf_valid_dataset, weights, biases, layers))
            print("Validation accuracy:", metrics.accuracy_score(np.argmax(y_valid, 1), np.argmax(prediction, 1)), "on step:", step)


In [4]:
num_steps = 30000
train_model(num_steps, weights, biases, optimizer, layers, run_optimization, calculate_prediction)

prediction = tf.nn.softmax(calculate_prediction(tf_test_dataset, weights, biases, layers))
deep_learning_accuracy = metrics.accuracy_score(np.argmax(y_test, 1), np.argmax(prediction, 1))
print("Точность на тестовых данных:", deep_learning_accuracy)

Validation accuracy: 0.1087 on step: 1
Validation accuracy: 0.1115 on step: 2
Validation accuracy: 0.1228 on step: 3
Validation accuracy: 0.1232 on step: 4
Validation accuracy: 0.138 on step: 5
Validation accuracy: 0.1375 on step: 6
Validation accuracy: 0.145 on step: 7
Validation accuracy: 0.1371 on step: 8
Validation accuracy: 0.1551 on step: 9
Validation accuracy: 0.8564 on step: 2000
Validation accuracy: 0.8708 on step: 4000
Validation accuracy: 0.878 on step: 6000
Validation accuracy: 0.8851 on step: 8000
Validation accuracy: 0.8857 on step: 10000
Validation accuracy: 0.8918 on step: 12000
Validation accuracy: 0.8917 on step: 14000
Validation accuracy: 0.8974 on step: 16000
Validation accuracy: 0.894 on step: 18000
Validation accuracy: 0.8955 on step: 20000
Validation accuracy: 0.8994 on step: 22000
Validation accuracy: 0.9002 on step: 24000
Validation accuracy: 0.8996 on step: 26000
Validation accuracy: 0.902 on step: 28000
Validation accuracy: 0.9027 on step: 30000
Точность на т

In [5]:
#2. Замените один из сверточных слоев на слой, реализующий операцию пулинга (Pooling) с функцией максимума или среднего. Как это повлияло на точность классификатора?

def calculate_prediction(tf_dataset, weigths, biases, layers):
    prev_layer = tf_dataset
    for index, layer in enumerate(layers[:-1]):
        conv = layer(tf.nn.conv2d(prev_layer, weigths[index], strides=[1, 1, 1, 1], padding='SAME') + biases[index])
        prev_layer = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding="SAME")

    shape = prev_layer.get_shape().as_list()
    reshape = tf.reshape(prev_layer, [shape[0], shape[1] * shape[2] * shape[3]])
    prev_layer = layers[-1](tf.matmul(reshape,  weigths[-2]) + biases[-2])

    return tf.matmul(prev_layer, weigths[-1]) + biases[-1]


In [6]:
layers = [tf.nn.relu, tf.nn.relu]
weights, biases = prepare_weights_biases(shape, num_labels, neurons, padding, stride, [deph, filter])

train_model(num_steps, weights, biases, optimizer, layers, run_optimization, calculate_prediction)

prediction = tf.nn.softmax(calculate_prediction(tf_test_dataset, weights, biases, layers))
deep_learning_accuracy = metrics.accuracy_score(np.argmax(y_test, 1), np.argmax(prediction, 1))
print("Точность на тестовых данных:", deep_learning_accuracy)

Validation accuracy: 0.1545 on step: 1
Validation accuracy: 0.1626 on step: 2
Validation accuracy: 0.19 on step: 3
Validation accuracy: 0.2003 on step: 4
Validation accuracy: 0.1834 on step: 5
Validation accuracy: 0.1977 on step: 6
Validation accuracy: 0.2547 on step: 7
Validation accuracy: 0.2868 on step: 8
Validation accuracy: 0.239 on step: 9
Validation accuracy: 0.8479 on step: 2000
Validation accuracy: 0.87 on step: 4000
Validation accuracy: 0.8761 on step: 6000
Validation accuracy: 0.8849 on step: 8000
Validation accuracy: 0.8823 on step: 10000
Validation accuracy: 0.8905 on step: 12000
Validation accuracy: 0.8904 on step: 14000
Validation accuracy: 0.8942 on step: 16000
Validation accuracy: 0.8943 on step: 18000
Validation accuracy: 0.8962 on step: 20000
Validation accuracy: 0.8984 on step: 22000
Validation accuracy: 0.8993 on step: 24000
Validation accuracy: 0.8979 on step: 26000
Validation accuracy: 0.9017 on step: 28000
Validation accuracy: 0.8995 on step: 30000
Точность на т

In [7]:
#3. Реализуйте классическую архитектуру сверточных сетей LeNet-5

def get_final_size(input_size, padding, stride, filters):
    output = input_size
    for filter in filters:
        output = (output + 2 * padding - filter) / stride + 1
        output = (output + 2 * padding - 2) / 1 + 1
    return int(np.ceil(output))

def prepare_weights_biases(input_number, output_number, neuron_number, padding, stride, *args):
    weights = []
    biases = []
    previous_deph = 1
    stddev=0.1

    for deph, filter in args:
        weights.append(tf.Variable(tf.random.truncated_normal([filter, filter, previous_deph, deph], stddev=stddev)))
        biases.append(tf.Variable(tf.zeros([deph])))
        previous_deph = deph

    final_image_size = get_final_size(input_number, padding, stride, map(lambda arg: arg[1], args))
    weights.append(tf.Variable(tf.random.truncated_normal([final_image_size * final_image_size * previous_deph, neuron_number[0]], stddev=stddev)))
    biases.append(tf.Variable(tf.zeros([neuron_number[0]])))

    weights.append(tf.Variable(tf.random.truncated_normal([neuron_number[0], neuron_number[1]], stddev=stddev)))
    biases.append(tf.Variable(tf.zeros([neuron_number[1]])))

    weights.append(tf.Variable(tf.random.truncated_normal([neuron_number[1], output_number], stddev=stddev)))
    biases.append(tf.Variable(tf.zeros([output_number])))

    return weights, biases

def calculate_prediction_dropout(tf_dataset, weigths, biases, layers):
    prev_layer = tf_dataset
    keep_prob = 0.1
    for index, layer in enumerate(layers[:-2]):
        conv = layer(tf.nn.conv2d(prev_layer, weigths[index], strides=[1, 1, 1, 1], padding='VALID') + biases[index])
        prev_layer = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

    shape = prev_layer.get_shape().as_list()
    reshape = tf.reshape(prev_layer, [shape[0], shape[1] * shape[2] * shape[3]])
    neuron_Layer = layers[-2](tf.matmul(reshape,  weigths[-3]) + biases[-3])
    prev_layer = tf.nn.dropout(neuron_Layer, keep_prob)

    neuron_Layer = layers[-1](tf.matmul(prev_layer,  weigths[-2]) + biases[-2])
    prev_layer = tf.nn.dropout(neuron_Layer, keep_prob)

    return tf.matmul(prev_layer, weigths[-1]) + biases[-1]

def calculate_prediction(tf_dataset, weigths, biases, layers):
    prev_layer = tf_dataset
    for index, layer in enumerate(layers[:-2]):
        conv = layer(tf.nn.conv2d(prev_layer, weigths[index], strides=[1, 1, 1, 1], padding='VALID') + biases[index])
        prev_layer = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding="VALID")

    shape = prev_layer.get_shape().as_list()
    reshape = tf.reshape(prev_layer, [shape[0], shape[1] * shape[2] * shape[3]])
    prev_layer = layers[-2](tf.matmul(reshape,  weigths[-3]) + biases[-3])

    prev_layer = layers[-1](tf.matmul(prev_layer,  weigths[-2]) + biases[-2])

    return tf.matmul(prev_layer, weigths[-1]) + biases[-1]

In [9]:
padding = 0
neuron_number = [100, 60]
layers = [tf.nn.relu, tf.nn.relu, tf.nn.relu, tf.nn.relu]
learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(start_learning_rate, 5000, 0.96, staircase=True)
optimizer = tf.optimizers.SGD(learning_rate)

weights, biases = prepare_weights_biases(shape, num_labels, neuron_number, padding, stride, [16, filter], [16, filter])

train_model(num_steps, weights, biases, optimizer, layers, run_optimization, calculate_prediction_dropout)

prediction = tf.nn.softmax(calculate_prediction(tf_test_dataset, weights, biases, layers))
deep_learning_accuracy = metrics.accuracy_score(np.argmax(y_test, 1), np.argmax(prediction, 1))
print("Точность на тестовых данных:", deep_learning_accuracy)

Validation accuracy: 0.1297 on step: 1
Validation accuracy: 0.1413 on step: 2
Validation accuracy: 0.1403 on step: 3
Validation accuracy: 0.1304 on step: 4
Validation accuracy: 0.1157 on step: 5
Validation accuracy: 0.119 on step: 6
Validation accuracy: 0.1239 on step: 7
Validation accuracy: 0.1471 on step: 8
Validation accuracy: 0.1422 on step: 9
Validation accuracy: 0.8266 on step: 2000
Validation accuracy: 0.8559 on step: 4000
Validation accuracy: 0.8682 on step: 6000
Validation accuracy: 0.8655 on step: 8000
Validation accuracy: 0.8706 on step: 10000
Validation accuracy: 0.8744 on step: 12000
Validation accuracy: 0.8728 on step: 14000
Validation accuracy: 0.8775 on step: 16000
Validation accuracy: 0.8791 on step: 18000
Validation accuracy: 0.8836 on step: 20000
Validation accuracy: 0.8736 on step: 22000
Validation accuracy: 0.8827 on step: 24000
Validation accuracy: 0.8881 on step: 26000
Validation accuracy: 0.8851 on step: 28000
Validation accuracy: 0.8818 on step: 30000
Точность 

In [10]:
#4. Сравните максимальные точности моделей, построенных в лабораторных работах 1-3. Как можно объяснить полученные различия?

print("Сверточная нейронная сеть:", deep_learning_accuracy)
print("Полносвязная нейронная сеть:", 0.9307306131168553)
print("Логистическая регрессия:", 0.8356)

Сверточная нейронная сеть: 0.9503311258278145
Полносвязная нейронная сеть: 0.9307306131168553
Логистическая регрессия: 0.8356
